# Setup SingleStore DDLs

Create and use the database llm_webinar

In [3]:
%%sql
CREATE DATABASE llm_webinar;

Create tables

In [14]:
%%sql

CREATE TABLE newsSentiment (
    title TEXT CHARACTER SET utf8mb4,
    url TEXT,
    time_published DATETIME,
    authors TEXT,
    summary TEXT CHARACTER SET utf8mb4,
    banner_image TEXT,
    source TEXT,
    category_within_source TEXT,
    source_domain TEXT,
    topic TEXT,
    topic_relevance_score TEXT,
    overall_sentiment_score REAL,
    overall_sentiment_label TEXT,
    `ticker` varchar(20) CHARACTER SET utf8 COLLATE utf8_general_ci DEFAULT NULL,
    ticker_relevance_score DECIMAL(10, 6),
    ticker_sentiment_score DECIMAL(10, 6),
    ticker_sentiment_label TEXT,
    SORT KEY (`ticker`,`time_published` DESC),
    SHARD KEY `__SHARDKEY` (`ticker`,`time_published` DESC),
    KEY(ticker) USING HASH,
    KEY(authors) USING HASH,
    KEY(source) USING HASH,
    KEY(overall_sentiment_label) USING HASH,
    KEY(ticker_sentiment_label) USING HASH
);

CREATE ROWSTORE REFERENCE TABLE companyInfo (
    ticker VARCHAR(10) PRIMARY KEY,
    AssetType VARCHAR(50),
    Name VARCHAR(100),
    Description TEXT,
    CIK VARCHAR(10),
    Exchange VARCHAR(10),
    Currency VARCHAR(10),
    Country VARCHAR(50),
    Sector VARCHAR(50),
    Industry VARCHAR(250),
    Address VARCHAR(100),
    FiscalYearEnd VARCHAR(20),
    LatestQuarter DATE,
    MarketCapitalization BIGINT,
    EBITDA BIGINT,
    PERatio DECIMAL(10, 2),
    PEGRatio DECIMAL(10, 3),
    BookValue DECIMAL(10, 2),
    DividendPerShare DECIMAL(10, 2),
    DividendYield DECIMAL(10, 4),
    EPS DECIMAL(10, 2),
    RevenuePerShareTTM DECIMAL(10, 2),
    ProfitMargin DECIMAL(10, 4),
    OperatingMarginTTM DECIMAL(10, 4),
    ReturnOnAssetsTTM DECIMAL(10, 4),
    ReturnOnEquityTTM DECIMAL(10, 4),
    RevenueTTM BIGINT,
    GrossProfitTTM BIGINT,
    DilutedEPSTTM DECIMAL(10, 2),
    QuarterlyEarningsGrowthYOY DECIMAL(10, 3),
    QuarterlyRevenueGrowthYOY DECIMAL(10, 3),
    AnalystTargetPrice DECIMAL(10, 2),
    TrailingPE DECIMAL(10, 2),
    ForwardPE DECIMAL(10, 2),
    PriceToSalesRatioTTM DECIMAL(10, 3),
    PriceToBookRatio DECIMAL(10, 2),
    EVToRevenue DECIMAL(10, 3),
    EVToEBITDA DECIMAL(10, 2),
    Beta DECIMAL(10, 3),
    52WeekHigh DECIMAL(10, 2),
    52WeekLow DECIMAL(10, 2),
    50DayMovingAverage DECIMAL(10, 2),
    200DayMovingAverage DECIMAL(10, 2),
    SharesOutstanding BIGINT,
    DividendDate DATE,
    ExDividendDate DATE
);


++
||
++
++

In [15]:
%%sql
SHOW TABLES;

Tables_in_stock_pull
companyInfo
embeddings
newsSentiment
stockTable


<a name="imports"></a>
- [Back to Contents](#contents)
# Install packages and imports

In [23]:
!pip install singlestoredb --quiet

In [25]:
import requests
import time
from datetime import datetime, timedelta
from datetime import datetime
import datetime
from dateutil.relativedelta import relativedelta
import singlestoredb as s2
import getpass

### Set API keys

In [26]:
alpha_vantage_apikey = getpass.getpass("enter alphavantage apikey here")

enter alphavantage apikey here ········


<a name="ingest"></a>
- [Back to Contents](#contents)
# Ingest from data sources

### Bring past two months of stock data

In [30]:
# set up connection to SingleStore and the ticker list
s2_conn = s2.connect(connection_url)
ticker_list = ['TSLA', 'AMZN', 'PLTR']

In [40]:
from datetime import datetime

def get_past_months(num_months):
    today = datetime.today()
    months = []

    for months_ago in range(0, num_months):
        target_date = today - relativedelta(months=months_ago)
        months.append(target_date.strftime('%Y-%m'))

    return months

num_months = 2  # Number of months
year_month_list = get_past_months(num_months)
print(year_month_list)

# pull intraday data for each stock and write to SingleStore
for ticker in ticker_list:
    print(ticker)
    data_list = []
    for year_month in year_month_list:
        print(year_month)

        intraday_price_url = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=5min&month={}&outputsize=full&apikey={}".format(ticker, year_month, alpha_vantage_apikey)
        r = requests.get(intraday_price_url)

        try:
            data = r.json()['Time Series (5min)']
        except:
            time.sleep(1) # required to not hit API limits
            continue

        for key in data:
            document = data[key]
            document['datetime'] = key
            document['ticker'] = ticker

            document['open'] = document['1. open']
            document['high'] = document['2. high']
            document['low'] = document['3. low']
            document['close'] = document['4. close']
            document['volume'] = document['5. volume']

            document['open'] = float(document['open'])
            document['high'] = float(document['high'])
            document['low'] = float(document['low'])
            document['close'] = float(document['close'])
            document['volume'] = int(document['volume'])


            del document['1. open']
            del document['2. high']
            del document['3. low']
            del document['4. close']
            del document['5. volume']

            data_list += [document]

            # Inside your loop, create the params dictionary with the correct values
            params = {
                'datetime': document['datetime'],
                'ticker': ticker,
                'open': document['open'],
                'high': document['high'],
                'low': document['low'],
                'close': document['close'],
                'volume': document['volume']
            }

            # Construct and execute the SQL statement
            table_name = 'stockTable'
            stmt = f"INSERT INTO {table_name} (created_at, ticker, open, high, low, close, volume) VALUES (%(datetime)s, %(ticker)s, %(open)s, %(high)s, %(low)s, %(close)s, %(volume)s)"

            with s2_conn.cursor() as cur:
                cur.execute(stmt, params)
        # time.sleep(1) # required to not hit API limits

['2023-10', '2023-09']
TSLA
2023-10
2023-09
AMZN
2023-10
2023-09
PLTR
2023-10
2023-09


In [14]:
%%sql
select count(*) from stockTable

count(*)
19126


# Bring in Company data

In [32]:
# pull company data for each stock and write to SingleStore
for ticker in ticker_list:
    print(ticker)
    data_list = []
    # for year_month in year_month_list:

    company_overview = "https://www.alphavantage.co/query?function=OVERVIEW&symbol={}&outputsize=full&apikey={}".format(ticker, alpha_vantage_apikey)
    r = requests.get(company_overview)

    try:
        data = r.json()
    except:
        time.sleep(3) # required to not hit API limits
        continue

    data['CIK'] = int(data['CIK'])
    data['MarketCapitalization']= float(data['MarketCapitalization'])
    # Assuming data['EBITDA'] is a string containing 'None'
    ebitda_str = data['EBITDA']
    if ebitda_str.lower() == 'none':
        # Handle the case where EBITDA is 'None', for example, you can set it to 0
        data['EBITDA'] = 0.0
    else:
        # Convert the EBITDA string to a float
        data['EBITDA'] = float(ebitda_str)

    PERatio_flt = data['PERatio']
    if PERatio_flt.lower() == 'none':
        # Handle the case where EVToRevenue is '-'
        data['PERatio'] = 0.0  # You can use any default value that makes sense
    else:
        # Convert the EVToRevenue string to a float
        data['PERatio'] = float(PERatio_flt)

    data['PEGRatio']= float(data['PEGRatio'])
    data['BookValue']= float(data['BookValue'])
    data['DividendPerShare']= float(data['DividendPerShare'])
    data['DividendYield']= float(data['DividendYield'])
    data['EPS']= float(data['EPS'])
    data['RevenuePerShareTTM']= float(data['RevenuePerShareTTM'])
    data['ProfitMargin']= float(data['ProfitMargin'])
    data['OperatingMarginTTM']= float(data['OperatingMarginTTM'])
    data['ReturnOnAssetsTTM']= float(data['ReturnOnAssetsTTM'])
    data['ReturnOnEquityTTM']= float(data['ReturnOnEquityTTM'])
    data['RevenueTTM']= int(data['RevenueTTM'])
    data['GrossProfitTTM']= int(data['GrossProfitTTM'])
    data['DilutedEPSTTM']= float(data['DilutedEPSTTM'])
    data['QuarterlyEarningsGrowthYOY']= float(data['QuarterlyEarningsGrowthYOY'])
    data['QuarterlyRevenueGrowthYOY']= float(data['QuarterlyRevenueGrowthYOY'])
    data['AnalystTargetPrice']= float(data['AnalystTargetPrice'])
    # Assuming data['TrailingPE'] is a string containing '-'
    trailing_pe_str = data['TrailingPE']
    if trailing_pe_str == '-':
        # Handle the case where TrailingPE is '-'
        data['TrailingPE'] = 0.0  # You can use any default value that makes sense
    else:
        try:
            # Attempt to convert the TrailingPE string to a float
            data['TrailingPE'] = float(trailing_pe_str)
        except ValueError:
            # Handle the case where the conversion fails (e.g., if it contains invalid characters)
            data['TrailingPE'] = 0.0  # Set to a default value or handle as needed



    data['ForwardPE']= float(data['ForwardPE'])
    data['PriceToSalesRatioTTM']= float(data['PriceToSalesRatioTTM'])
    # Assuming data['EVToRevenue'] is a string containing '-'
    PriceToBookRatio_flt = data['PriceToBookRatio']
    if PriceToBookRatio_flt == '-':
        # Handle the case where EVToRevenue is '-'
        data['PriceToBookRatio'] = 0.0  # You can use any default value that makes sense
    else:
        # Convert the EVToRevenue string to a float
        data['PriceToBookRatio'] = float(PriceToBookRatio_flt)


    # Assuming data['EVToRevenue'] is a string containing '-'
    ev_to_revenue_str = data['EVToRevenue']
    if ev_to_revenue_str == '-':
        # Handle the case where EVToRevenue is '-'
        data['EVToRevenue'] = 0.0  # You can use any default value that makes sense
    else:
        # Convert the EVToRevenue string to a float
        data['EVToRevenue'] = float(ev_to_revenue_str)

    # data['EVToEBITDA']= float(data['EVToEBITDA'])
    # Assuming data['EVToRevenue'] is a string containing '-'
    ev_to_EBITDA_str = data['EVToEBITDA']
    if ev_to_revenue_str == '-':
        # Handle the case where EVToRevenue is '-'
        data['EVToEBITDA'] = 0.0  # You can use any default value that makes sense
    else:
        # Convert the EVToRevenue string to a float
        data['EVToEBITDA'] = float(ev_to_EBITDA_str)


    data['Beta']= float(data['Beta'])
    data['52WeekHigh']= float(data['52WeekHigh'])
    data['52WeekLow']= float(data['52WeekLow'])
    data['50DayMovingAverage']= float(data['50DayMovingAverage'])
    data['200DayMovingAverage']= float(data['200DayMovingAverage'])
    data['SharesOutstanding']= int(data['SharesOutstanding'])
    # description_embedding = [np.array(x, '<f4') for x in get_embeddings(data["Description"], engine=model)]
    dividend_date_str = data['DividendDate']
    if dividend_date_str.lower() == 'none':
        # Handle the case where EBITDA is 'None', for example, you can set it to 0
        data['DividendDate'] = '9999-12-31'
    else:
        # Convert the EBITDA string to a float
        data['DividendDate'] = str(dividend_date_str)


    exdividend_date_str = data['ExDividendDate']
    if exdividend_date_str.lower() == 'none':
        # Handle the case where EBITDA is 'None', for example, you can set it to 0
        data['ExDividendDate'] = '9999-12-31'
    else:
        # Convert the EBITDA string to a float
        data['ExDividendDate'] = str(exdividend_date_str)


    data_list += [data]

        # Inside your loop, create the params dictionary with the correct values
    params = {
                "Symbol": data["Symbol"],
                "AssetType": data["AssetType"],
                "Name": data["Name"],
                "Description": data["Description"],
                "CIK": data["CIK"],
                "Exchange": data["Exchange"],
                "Currency": data["Currency"],
                "Country": data["Country"],
                "Sector": data["Sector"],
                "Industry": data["Industry"],
                "Address": data["Address"],
                "FiscalYearEnd": data["FiscalYearEnd"],
                "LatestQuarter": data["LatestQuarter"],
                "MarketCapitalization": data["MarketCapitalization"],
                "EBITDA": data["EBITDA"],
                "PERatio": data["PERatio"],
                "PEGRatio": data["PEGRatio"],
                "BookValue": data["BookValue"],
                "DividendPerShare": data["DividendPerShare"],
                "DividendYield": data["DividendYield"],
                "EPS": data["EPS"],
                "RevenuePerShareTTM": data["RevenuePerShareTTM"],
                "ProfitMargin": data["ProfitMargin"],
                "OperatingMarginTTM": data["OperatingMarginTTM"],
                "ReturnOnAssetsTTM": data["ReturnOnAssetsTTM"],
                "ReturnOnEquityTTM": data["ReturnOnEquityTTM"],
                "RevenueTTM": data["RevenueTTM"],
                "GrossProfitTTM": data["GrossProfitTTM"],
                "DilutedEPSTTM": data["DilutedEPSTTM"],
                "QuarterlyEarningsGrowthYOY": data["QuarterlyEarningsGrowthYOY"],
                "QuarterlyRevenueGrowthYOY": data["QuarterlyRevenueGrowthYOY"],
                "AnalystTargetPrice": data["AnalystTargetPrice"],
                "TrailingPE": data["TrailingPE"],
                "ForwardPE": data["ForwardPE"],
                "PriceToSalesRatioTTM": data["PriceToSalesRatioTTM"],
                "PriceToBookRatio": data["PriceToBookRatio"],
                "EVToRevenue": data["EVToRevenue"],
                "EVToEBITDA": data["EVToEBITDA"],
                "Beta": data["Beta"],
                "52WeekHigh": data["52WeekHigh"],
                "52WeekLow": data["52WeekLow"],
                "50DayMovingAverage": data["50DayMovingAverage"],
                "200DayMovingAverage": data["200DayMovingAverage"],
                "SharesOutstanding": data["SharesOutstanding"],
                "DividendDate": data["DividendDate"],
                "ExDividendDate": data["ExDividendDate"]
    }

     # Construct and execute the SQL statement
    table_name = 'companyInfo'
    stmt = f"INSERT INTO {table_name} (ticker, AssetType, Name, Description, CIK, Exchange, Currency, Country, Sector, Industry, Address, FiscalYearEnd, LatestQuarter, MarketCapitalization, EBITDA, PERatio, PEGRatio, BookValue, DividendPerShare, DividendYield, EPS, RevenuePerShareTTM, ProfitMargin, OperatingMarginTTM, ReturnOnAssetsTTM, ReturnOnEquityTTM, RevenueTTM, GrossProfitTTM, DilutedEPSTTM, QuarterlyEarningsGrowthYOY, QuarterlyRevenueGrowthYOY, AnalystTargetPrice, TrailingPE, ForwardPE, PriceToSalesRatioTTM, PriceToBookRatio, EVToRevenue, EVToEBITDA, Beta, 52WeekHigh, 52WeekLow, 50DayMovingAverage, 200DayMovingAverage, SharesOutstanding, DividendDate, ExDividendDate) VALUES (%(Symbol)s, %(AssetType)s, %(Name)s, %(Description)s, %(CIK)s, %(Exchange)s, %(Currency)s, %(Country)s, %(Sector)s, %(Industry)s, %(Address)s, %(FiscalYearEnd)s, %(LatestQuarter)s, %(MarketCapitalization)s, %(EBITDA)s, %(PERatio)s, %(PEGRatio)s, %(BookValue)s, %(DividendPerShare)s, %(DividendYield)s, %(EPS)s, %(RevenuePerShareTTM)s, %(ProfitMargin)s, %(OperatingMarginTTM)s, %(ReturnOnAssetsTTM)s, %(ReturnOnEquityTTM)s, %(RevenueTTM)s, %(GrossProfitTTM)s, %(DilutedEPSTTM)s, %(QuarterlyEarningsGrowthYOY)s, %(QuarterlyRevenueGrowthYOY)s, %(AnalystTargetPrice)s, %(TrailingPE)s, %(ForwardPE)s, %(PriceToSalesRatioTTM)s, %(PriceToBookRatio)s, %(EVToRevenue)s, %(EVToEBITDA)s, %(Beta)s, %(52WeekHigh)s, %(52WeekLow)s, %(50DayMovingAverage)s, %(200DayMovingAverage)s, %(SharesOutstanding)s, %(DividendDate)s, %(ExDividendDate)s)"

# Replace table_name with the actual table name you're using.
    with s2_conn.cursor() as cur:
        cur.execute(stmt, params)

TSLA
AMZN
PLTR


In [33]:
%%sql
select * from companyInfo limit 1

ticker,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
AMZN,Common Stock,Amazon.com Inc,"Amazon.com, Inc. is an American multinational technology company which focuses on e-commerce, cloud computing, digital streaming, and artificial intelligence. It is one of the Big Five companies in the U.S. information technology industry, along with Google, Apple, Microsoft, and Facebook. The company has been referred to as one of the most influential economic and cultural forces in the world, as well as the world's most valuable brand.",1018724,NASDAQ,USD,USA,TRADE & SERVICES,RETAIL-CATALOG & MAIL-ORDER HOUSES,"410 TERRY AVENUE NORTH, SEATTLE, WA, US",December,2023-06-30,1322019848000,63431000000,101.69,11.850,16.35,0.00,0.0104,1.26,52.56,0.0243,0.0572,0.0247,0.0872,538046005000,225152000000,1.26,-0.986,0.108,170.30,101.69,43.86,1.797,6.61,1.948,19.30,1.188,145.86,81.43,133.88,115.95,10317800000,9999-12-31,9999-12-31


# Bring in news sentiment

In [ ]:
import datetime
# pull news sentiment data for each stock and write to SingleStore
for ticker in ticker_list:
    print(ticker)
    data_list = []


    for i in year_month_list:
        date_object = datetime.datetime.strptime(i, '%Y-%m')
        print(date_object)
        output_date = date_object.strftime('%Y%m%d') + "T0000"

        # Get the next month from the 'date_object'
        previous_month_date = date_object + relativedelta(months=-1)
        previous_month_date = previous_month_date.strftime('%Y%m%d') + "T0000"

        # Update 'date_object' for the next iteration
        date_object = previous_month_date

        # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
        news_and_sentiment = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={}&time_from={}&time_to={}&limit=1000&outputsize=full&apikey={}'.format(ticker, previous_month_date, output_date, alpha_vantage_apikey)
        r = requests.get(news_and_sentiment)

        try:
            data = r.json()
            data = data["feed"]
        except:
            time.sleep(2) # required to not hit API limits
            continue


        for item in data:
            item['title'] = str(item['title'])
            item['url'] = str(item['url'])
            item['time_published'] = datetime.datetime.strptime(str(item['time_published']), "%Y%m%dT%H%M%S").strftime("%Y-%m-%d %H:%M:%S")

            if item['authors']:
                # Check if the 'authors' list is not empty
                authors_str = str(item['authors'][0])
            else:
                # Handle the case where 'authors' is empty
                authors_str = "No authors available"

            item['authors'] = authors_str

            item['summary'] = str(item['summary'])
            item['banner_image'] = str(item['banner_image'])
            item['source'] = str(item['source'])
            item['category_within_source'] = str(item['category_within_source'])
            item['source_domain'] = str(item['source_domain'])
            item['topic'] = str(item['topics'][0]["topic"])
            item['topic_relevance_score'] = float(item['topics'][0]['relevance_score'])
            item['overall_sentiment_score'] = float(item['overall_sentiment_score'])
            item['overall_sentiment_label'] = str(item['overall_sentiment_label'])
            item['ticker'] = str(item['ticker_sentiment'][0]['ticker'])
            item['ticker_relevance_score'] = float(item['ticker_sentiment'][0]['relevance_score'])
            item['ticker_sentiment_score'] = float(item['ticker_sentiment'][0]['ticker_sentiment_score'])
            item['ticker_sentiment_label'] = str(item['ticker_sentiment'][0]['ticker_sentiment_label'])


            params= {
                "title": item["title"],
                "url": item["url"],
                "time_published": item["time_published"],
                "authors": item["authors"],
                "summary": item["summary"],
                "banner_image": item["banner_image"],
                "source": item["source"],
                "category_within_source": item["category_within_source"],
                "source_domain": item["source_domain"],
                "topic": item["topic"],
                "topic_relevance_score": item['topic_relevance_score'],
                'overall_sentiment_score': item['overall_sentiment_score'],
                'overall_sentiment_label': item['overall_sentiment_label'],
                'ticker': item['ticker'],
                'ticker_relevance_score': item['ticker_relevance_score'],
                'ticker_sentiment_score': item['ticker_sentiment_score'],
                'ticker_sentiment_label': item['ticker_sentiment_label']
            }
            #print(params)


             # Construct and execute the SQL statement
            table_name = 'newsSentiment'
            stmt = f"INSERT INTO {table_name} (title, url, time_published, authors, summary, banner_image, source, category_within_source, source_domain, topic, topic_relevance_score, overall_sentiment_score, overall_sentiment_label, ticker, ticker_relevance_score, ticker_sentiment_score, ticker_sentiment_label) VALUES (%(title)s, %(url)s, %(time_published)s, %(authors)s, %(summary)s, %(banner_image)s, %(source)s, %(category_within_source)s, %(source_domain)s, %(topic)s, %(topic_relevance_score)s, %(overall_sentiment_score)s, %(overall_sentiment_label)s, %(ticker)s, %(ticker_relevance_score)s, %(ticker_sentiment_score)s, %(ticker_sentiment_label)s)"

            # Replace table_name with the actual table name you're using.

            with s2_conn.cursor() as cur:
                cur.execute(stmt, params)

TSLA
2023-10-01 00:00:00
2023-09-01 00:00:00
AMZN
2023-10-01 00:00:00
2023-09-01 00:00:00


In [15]:
%%sql
SELECT count(*) Rows_in_newsSentiment FROM newsSentiment